# Baseline 알고리즘 기본 코드

사용자 Bias와 아이템 Bias를 고려

## #01. 준비작업

### [1] 패키지 가져오기

In [93]:
import warnings
warnings.filterwarnings('ignore')

# Intel SKlearn 하드웨어 가속 패치 설정
import sys
if sys.platform == 'win32':
    from sklearnex import patch_sklearn
    patch_sklearn()

from hossam.util import *
from hossam.plot import *
from hossam.analysis import *

from surprise import Reader, Dataset, BaselineOnly
from surprise.model_selection import train_test_split, cross_validate, GridSearchCV, RandomizedSearchCV
from surprise.accuracy import rmse, mae

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


### [2] 데이터 가져오기

In [94]:
origin = my_read_excel("https://data.hossam.kr/mldata/movie_ratings.xlsx", sheet_name='ratings')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB
None

데이터프레임 상위 5개 행
+----+----------+-----------+----------+-------------+
|    |   userId |   movieId |   rating |   timestamp |
|----+----------+-----------+----------+-------------|
|  0 |        1 |         1 |        4 | 9.64983e+08 |
|  1 |        1 |         3 |        4 | 9.64981e+08 |
|  2 |        1 |         6 |        4 | 9.64982e+08 |
|  3 |        1 |        47 |        5 | 9.64984e+08 |
|  4 |        1 |        50 |        5 | 9.64983e+08 |
+----+----------+-----------+----------+-------------+

데이터프레임 하위 5개 행
+--------+----------+-----------+----------+-------------+
|  

## #02. 데이터 전처리

### [1] Surprise 형식의 데이터로 변환

`사용자 번호, 아이템 번호, 평점` 구조의 데이터를 만족해야 한다.

In [95]:
df = origin.drop('timestamp', axis=1)

# 평점의 분포를 알려준다.
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(df, reader)
data

## #03. 추천 모형 구현

### [1] 기본 코드

#### (1) 훈련, 검증 데이터 분리

sklearn이 아닌 surprise 자체 함수 사용

In [96]:
train, test = train_test_split(data, test_size=0.2, random_state=1234)

#### (2) 추천 모형 학습 후 성능 평가

In [97]:
estimator = BaselineOnly()
estimator.fit(train)
pred = estimator.test(test)
pred[:5]

Estimating biases using als...


[Prediction(uid=603, iid=3996, r_ui=5.0, est=3.865470094018238, details={'was_impossible': False}),
 Prediction(uid=199, iid=2912, r_ui=4.0, est=3.5270186068257785, details={'was_impossible': False}),
 Prediction(uid=416, iid=2716, r_ui=2.0, est=3.2531312891488335, details={'was_impossible': False}),
 Prediction(uid=589, iid=150, r_ui=4.0, est=4.143871102075767, details={'was_impossible': False}),
 Prediction(uid=307, iid=6755, r_ui=4.0, est=2.6344308636371943, details={'was_impossible': False})]

> uid: 사용자 번호, iid: 아이템 번호, r_ui: 해당 사용자가 실제로 부여한 평점, est: 예측평점

#### (3) 특정 유저가 특정 영화에 부여할 평점 예상

In [98]:
upred = estimator.predict(uid=603, iid=3996)
upred

Prediction(uid=603, iid=3996, r_ui=None, est=3.865470094018238, details={'was_impossible': False})

In [99]:
upred.est

3.865470094018238

#### (3) 성능평가

In [100]:
rmse(pred), mae(pred)

RMSE: 0.8715
MAE:  0.6706


(0.8715309792778995, 0.6706040327595953)

### [2] 교차검증

#### (1) 교차검증을 위한 하이퍼파라미터 설정

In [101]:
estimator = BaselineOnly(bsl_options={
    "method": "als",    # 알고리즘 "als" or "sgd"
    "n_epochs": 10,     # 반복횟수 (기본값=10)
    "reg_u": 10,        # 항목에 대한 정규화 매개변수 (기본값=10)
    "reg_i": 15         # 사용자를 위한 정규화 매개변수 (기본값=15)
})
cv_result = cross_validate(estimator, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8762  0.8797  0.8795  0.8767  0.8696  0.8763  0.0037  
MAE (testset)     0.6738  0.6785  0.6797  0.6759  0.6729  0.6762  0.0026  
Fit time          0.18    0.19    0.20    0.20    0.20    0.19    0.01    
Test time         0.04    0.04    0.04    0.04    0.04    0.04    0.00    


#### (2) 교차검증 결과 확인

In [102]:
cv_result

{'test_rmse': array([0.87616071, 0.87973344, 0.87953172, 0.87667077, 0.86963703]),
 'test_mae': array([0.67381834, 0.67850223, 0.67971106, 0.67586788, 0.6728928 ]),
 'fit_time': (0.18199968338012695,
  0.192002534866333,
  0.1960010528564453,
  0.19699764251708984,
  0.19899702072143555),
 'test_time': (0.04099869728088379,
  0.04099774360656738,
  0.041031599044799805,
  0.04400205612182617,
  0.04099917411804199)}

#### (3) 교차검증 성능 평가 지표 출력

In [103]:
print("RMSE(mean):", cv_result['test_rmse'].mean())
print("MAE(mean):", cv_result['test_mae'].mean())

RMSE(mean): 0.8763467350072563
MAE(mean): 0.6761584620297962


### [3] 하이퍼파라미터 튜닝

#### (1) 학습 모형 구성

In [104]:
params = {
    'bsl_options': {
        "method": ["als", "sgd"],    # 알고리즘 "als" or "sgd"
        "n_epochs": [10, 20],        # 반복횟수 (기본값=10)
        "reg_u": [10, 12],           # 사용자에 대한 정규화 매개변수 (기본값=10)
        "reg_i": [15, 20]            # 아이템에 대한 정규화 매개변수 (기본값=15)
    }
}

# grid = GridSearchCV(BaselineOnly, 
#                     param_grid=params, 
#                     measures=['RMSE', 'MAE'], 
#                     cv=5, 
#                     n_jobs=-1)
                    
grid = RandomizedSearchCV(BaselineOnly, 
                        param_distributions=params, 
                        measures=['RMSE', 'MAE'], 
                        cv=5, 
                        n_jobs=-1, 
                        random_state=1234)

grid.fit(data)

#### (2) 성능 평가 지표 확인

In [105]:
grid.best_score

{'rmse': 0.8693399143463537, 'mae': 0.6680779326654303}

#### (3) 최적 하이퍼파라미터 확인

In [106]:
grid.best_params

{'rmse': {'bsl_options': {'method': 'sgd',
   'n_epochs': 20,
   'reg_u': 10,
   'reg_i': 15}},
 'mae': {'bsl_options': {'method': 'sgd',
   'n_epochs': 20,
   'reg_u': 10,
   'reg_i': 15}}}

In [110]:
grid.cv_results

{'split0_test_rmse': array([0.86813757, 0.87228505, 0.86835339, 0.87249151, 0.86812934,
        0.87228119, 0.86834602, 0.87248806, 0.86892136, 0.86892136,
        0.86892136, 0.86892136, 0.86065042, 0.86065042, 0.86065042,
        0.86065042]),
 'split1_test_rmse': array([0.87673191, 0.88047836, 0.87690596, 0.88063861, 0.87672207,
        0.88047386, 0.87689724, 0.88063463, 0.87745158, 0.87745158,
        0.87745158, 0.87745158, 0.87055292, 0.87055292, 0.87055292,
        0.87055292]),
 'split2_test_rmse': array([0.87753038, 0.88153673, 0.87774535, 0.88174405, 0.87752167,
        0.88153271, 0.87773768, 0.8817405 , 0.87766255, 0.87766255,
        0.87766255, 0.87766255, 0.87025908, 0.87025908, 0.87025908,
        0.87025908]),
 'split3_test_rmse': array([0.87522201, 0.87918219, 0.87544556, 0.87939868, 0.87521238,
        0.87917782, 0.87543697, 0.87939478, 0.87547721, 0.87547721,
        0.87547721, 0.87547721, 0.8681453 , 0.8681453 , 0.8681453 ,
        0.8681453 ]),
 'split4_test_rm

#### (4) 최적 추정기

In [107]:
grid.best_estimator

{'rmse': <surprise.prediction_algorithms.baseline_only.BaselineOnly at 0x21d27daec50>,
 'mae': <surprise.prediction_algorithms.baseline_only.BaselineOnly at 0x21d1e186650>}

#### (5) 최적 추정기를 활용한 컨텐츠 추천

학습을 다시 시켜야 한다.

In [108]:
estimator = grid.best_estimator['rmse']
estimator.fit(train)
pred = estimator.test(test)
pred[:5]

Estimating biases using sgd...


[Prediction(uid=603, iid=3996, r_ui=5.0, est=3.9696392415813557, details={'was_impossible': False}),
 Prediction(uid=199, iid=2912, r_ui=4.0, est=3.548906012863939, details={'was_impossible': False}),
 Prediction(uid=416, iid=2716, r_ui=2.0, est=3.0954432297290766, details={'was_impossible': False}),
 Prediction(uid=589, iid=150, r_ui=4.0, est=4.267549985510884, details={'was_impossible': False}),
 Prediction(uid=307, iid=6755, r_ui=4.0, est=2.601641316126221, details={'was_impossible': False})]

In [109]:
estimator.predict(uid=603, iid=3996)

Prediction(uid=603, iid=3996, r_ui=None, est=3.9696392415813557, details={'was_impossible': False})